In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#project
from macroecon_wrappy import (
    Auth,
    Metric,
    Measure,
    Epoch,
    plots
)
from macroecon_wrappy.adapters import (
    FredApi,
    YahooFin
)
from macroecon_wrappy.extractors import (
    NberExtract
)

#external
from fredapi import Fred
import yfinance as yf

#system
from pathlib import Path

In [3]:
secrets_path = Path('/workspaces/API-macroecon-wrappy/SECRETS.yaml')
cache_path = Path('/workspaces/API-macroecon-wrappy/tests/tmp/')
auth = Auth(secrets_path, cache_path)
auth.load_secrets()

True

In [4]:
#fred
FredApi.set_wrapper(auth, Fred)
gdp = FredApi.get_data('GDP')
gdi = FredApi.get_data('GDI')
pop = FredApi.get_data('POPTHM')

In [5]:
#nber
NberExtract.set_config(auth)
name = 'recessions'
recessions = NberExtract.get_data(name)

first two columns will be renamed: ['start', 'end', 'name']


#yahoo
YahooFin.set_wrapper(auth, yf)
msft = YahooFin.get_data(tickers='MSFT')

In [6]:
output = Measure([gdp, gdi, pop], recessions)

In [7]:
output

                col-0  col-1     col-2
timestamp                             
2024-06-01        NaN    NaN  336839.0
2024-05-01        NaN    NaN  336687.0
2024-04-01  28629.153    NaN  336550.0
2024-03-01        NaN    NaN  336423.0
2024-02-01        NaN    NaN  336306.0

In [8]:
assert output.get_cycle().df().shape == (35,3)
assert isinstance(output, Measure) == True
gdp._metadata['units'], gdi._metadata['units'], pop._metadata['units']

('Billions of Dollars', 'Billions of Dollars', 'Thousands')

In [9]:
from macroecon_wrappy import plots
plots.graph_ts_js(output, cols=[], recession_bars=True, log_scale='y')

alt.VConcatChart(...)

In [13]:
output = Measure([gdp], recessions)
tmp = output.to_long_by_cycle()
tmp = tmp[tmp.grp=='col-0']
tmp

,timestamp,grp,value,diff_days,diff_pct,cycle
0,1949-07-01,col-0,0.799016,1369,0.937029,span-10.97months\t| 1948/11/1 - 1949/10/1
1,1949-04-01,col-0,0.757674,1278,0.874743,span-10.97months\t| 1948/11/1 - 1949/10/1
2,1949-01-01,col-0,0.856674,1188,0.813142,span-10.97months\t| 1948/11/1 - 1949/10/1
3,1948-10-01,col-0,1.000000,1096,0.750171,span-10.97months\t| 1948/11/1 - 1949/10/1
4,1948-07-01,col-0,0.968550,1004,0.687201,span-10.97months\t| 1948/11/1 - 1949/10/1
...,...,...,...,...,...,...
38,2010-07-01,col-0,0.092939,395,0.099823,span-1.97months\t| 2020/2/1 - 2020/4/1
39,2010-04-01,col-0,0.071283,304,0.076826,span-1.97months\t| 2020/2/1 - 2020/4/1
40,2010-01-01,col-0,0.042360,214,0.054081,span-1.97months\t| 2020/2/1 - 2020/4/1
41,2009-10-01,col-0,0.027151,122,0.030831,span-1.97months\t| 2020/2/1 - 2020/4/1


# Unintegrated

In [11]:
import pandas as pd
import altair as alt

### Graph Cycle

In [44]:
#graph_cycle

tmp['cycle_months'] = (tmp['cycle'].str.split('span-').str[1].str.split('months').str[0]).astype(float)
tmp.sort_values(by='cycle_months', inplace=True)

base = alt.Chart(tmp, width=600, height=200)
line = base.mark_line().encode(
                alt.X('diff_pct:Q'),
                alt.Y('value:Q'),
                color=alt.Color('cycle', legend=None),
                opacity=alt.value(0.25),
                tooltip='cycle:N'
            )
points = line.mark_point(filled=True).encode(
    color='cycle',
    #color=alt.Color('cycle', sort=['cycle_months']),     #sort the legend by months
    shape=alt.Shape('cycle', 
                    scale=alt.Scale(range=['cross', 'circle', 'square', 'triangle-right', 'diamond']),
                    #sort=['cycle_months']
                    )
)

#combines color, shape in legend
alt.layer(
    line,
    points
).resolve_scale(
    color='independent',
    shape='independent'
)

alt.LayerChart(...)

### Candlestick

In [45]:
#yahoo
YahooFin.set_wrapper(auth, yf)
msft = YahooFin.get_data(tickers='MSFT')
df = pd.DataFrame(msft, columns=['msft'])

[*********************100%%**********************]  1 of 1 completed


In [80]:
from pathlib import Path

file = Path('/workspaces/API-macroecon-wrappy/tests/data/msft.csv')
assert file.is_file()

df = pd.read_csv(file)
df['colors'] = ['green' if c>o else 'red' for c,o in zip(df.Close,df.Open)]
df.shape

True

In [106]:
import ta
df['SMA'] = ta.trend.sma_indicator(df.Close, window = 3)
df['BLH'] = ta.volatility.bollinger_hband(df.Close,3,)
df['BLL'] = ta.volatility.bollinger_lband(df.Close,3)

up = df[df.Close>=df.Open]
dwn = df[df.Close<df.Open]
c1, c2, c3, c4 = 'green', 'red', 'blue', 'gray'

In [104]:
#candlestick

import altair as alt
bind = alt.selection_interval(bind='scales') 

# Candlestick chart
base=alt.Chart(df).encode(
    x = alt.X('Dates:T', 
              axis=alt.Axis(title='Date')
    ),
    color=alt.condition(
        'datum.open <= datum.close',
        alt.value(c1), 
        alt.value(c2)
        )
    )
rule=base.mark_rule().encode(
    y = alt.Y(
        'Low:Q', 
        title='Price', 
        scale = alt.Scale(zero=False)
    ),
    y2 = alt.Y2('High:Q')
    )
bar=base.mark_bar(width=4).encode(
    y = alt.Y('Open:Q'),
    y2 = alt.Y2('Close:Q'),
    tooltip = ['Date','Open','High','Low','Close']
    ).properties(width=800,height=300)

# Adding technical analysis
sma = base.mark_line().encode(y = alt.Y('SMA:Q'), tooltip=['Date','SMA'],
                              color=alt.value(c3))
blh = base.mark_line().encode(y = alt.Y('BLH:Q'), tooltip=['Date','BLH'],
                              color=alt.value(c3))
bll = base.mark_line().encode(y = alt.Y('BLL:Q'), tooltip=['Date','BLL'],
                              color=alt.value(c3))
fill = base.mark_area(fillOpacity=0.1).encode(x='Date:T', y='BLL:Q', y2='BLH:Q',
                                              color=alt.value(c3))

candlestick = sma + blh + bll + fill + rule + bar
candlestick_b = candlestick.add_selection(bind)

# volume bars
volume = base.mark_bar(width=4).encode(
    y = alt.Y('Volume:Q'),
    tooltip = ['Volume']
    ).properties(width=800, height=150)
volume_b = volume.add_selection(bind)

combine = candlestick_b & volume_b
#combine.save('c_stick.png')       #<< for saving
combine

/home/vscode/.local/share/virtualenvs/API-macroecon-wrappy-Ov67JMZg/lib/python3.12/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.


alt.VConcatChart(...)

In [105]:
#Altair filtering by selection
interval = alt.selection_interval()

candlestick_i = candlestick.add_selection(interval)
volume_i = volume.transform_filter(interval)

combine = candlestick_i & volume_i
combine

/home/vscode/.local/share/virtualenvs/API-macroecon-wrappy-Ov67JMZg/lib/python3.12/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.


alt.VConcatChart(...)

### Line Chart

In [108]:
from pathlib import Path

file = Path('/workspaces/API-macroecon-wrappy/tests/data/msft_long.csv')
assert file.is_file()

ema_df = pd.read_csv(file)

#TODO:create 5 exp moving averages: 10,20,50,100,200

In [ ]:

#copy date from index
ema_df['date']=ema_df.index

#create long form dataframe
ema_df = ema_df.melt('date',var_name='line',value_name='price')

#plot line chart
line = alt.Chart(ema_df,title='Price Daily Chart').mark_line().encode(
    alt.X('date:T').title('Date'),
    alt.Y('price:Q',scale=alt.Scale(domain=[ema_df.price.min()*.8,ema_df.price.max()*1.2])),
     color=alt.Color('line:N', 
               legend=alt.Legend(orient='top',title='')),
    strokeWidth=alt.condition(
        "datum.line == 'Adj Close'",
        alt.value(2),
        alt.value(1))
  ).properties(
      width=600,
      height=400
      ) #set the size of chart

#find points
mark = ema_df.query('price==price.max()')
#plot points
point = alt.Chart(mark).mark_circle().encode(
    alt.X('date:T'),
    alt.Y('price:Q'),
    color='line:N',
    text=alt.Text("price:Q")
)
#plot text
text = alt.Chart(mark).mark_text(dy=-5,xOffset=20).encode(
    alt.X('date:T'),
    alt.Y('price:Q'),
    color='line:N',
    text=alt.Text("price:Q")
)

#render the full chart
line + point + text 